# RT PLAN READER

In [2]:
import dicom as dcm
import os
import numpy as np

In [3]:
plan=dcm.read_file('../interplay/data/planned//new/RTplan/RP_1D_SIN_ITV_35.dcm')

In [4]:
#print plan.BeamSequence[0].BlockSequence[0].BlockName
#print(plan)
print(plan.FractionGroupSequence[0])

(300a, 0071) Fraction Group Number               IS: '1'
(300a, 0078) Number of Fractions Planned         IS: '1'
(300a, 0080) Number of Beams                     IS: '1'
(300a, 00a0) Number of Brachy Application Setups IS: '0'
(300c, 0004)  Referenced Beam Sequence   1 item(s) ---- 
   (300a, 0084) Beam Dose                           DS: '2'
   (300a, 0086) Beam Meterset                       DS: '697.769853271069'
   (300c, 0006) Referenced Beam Number              IS: '1'
   (3249, 0010) Private Creator                     OB: Array of 34 bytes
   (3249, 1000) Private tag data                    OB: b'99'
   (3249, 1010) Private tag data                    OB: Array of 52 bytes
   ---------


In [5]:
#print(plan.FractionGroupSequence[0].ReferencedBeamSequence[3]) #[3])#BeamMeterset)
#print(plan.DoseReferenceSequence[0].DeliveryMaximumDose)
#print(plan.FractionGroupSequence[0].NumberofFractionsPlanned
print(plan.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode)

NON_STANDARD


## CREATE A PYTHON DICTIONARY OBJECT WITH USEFUL DATA OF RT PLAN

In [6]:
#Dictonary which will contains all useful data for RTplan
rt_plan_file = {} 

rt_plan_file['Name'] = plan.PatientName
rt_plan_file['ID'] = plan.PatientID
rt_plan_file['Birth'] = plan.PatientBirthDate

#Create a dictionnary for each beam in RT plan dictionnary
rt_plan_file['Beam'] = {}

for beam_ in plan.BeamSequence:
    
    if beam_.TreatmentDeliveryType == 'TREATMENT':
        
            beam_name=beam_.BeamName
            rt_plan_file['Beam'][beam_name] = {}  #Create a dictionnary for each (treatment Beam)
            rt_plan_file['Beam'][beam_name]['RadiationType'] = beam_.RadiationType
            Evalue=str(beam_.ControlPointSequence[0].NominalBeamEnergy)
            
            if rt_plan_file['Beam'][beam_name]['RadiationType'] == 'ELECTRON':
                fluence_mode='E'             
            else:
                if plan.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode == 'NON_STANDARD':
                    fluence_mode='FFF'
                else:
                    fluence_mode='X'
            energy=Evalue+fluence_mode
            rt_plan_file['Beam'][beam_name]['Energy']=energy
            rt_plan_file['Beam'][beam_name]['BeamType']=beam_.BeamType


            xj = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].LeafJawPositions
            yj = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions

            rt_plan_file['Beam'][beam_name]['x_jaw'] = [float(val) for val in xj]
            rt_plan_file['Beam'][beam_name]['y_jaw'] = [float(val) for val in yj]
            
            rt_plan_file['Beam'][beam_name]['iso']=[float(val) for val in beam_.ControlPointSequence[0].IsocenterPosition]
            rt_plan_file['Beam'][beam_name]['iso'][1]=rt_plan_file['Beam'][beam_name]['iso'][1]*-1  #because rotation (x axis) in GATE
            rt_plan_file['Beam'][beam_name]['iso'][2]=rt_plan_file['Beam'][beam_name]['iso'][2]*-1  #because rotation (x axis) in GATE
            
            rt_plan_file['Beam'][beam_name]['rot_colli'] = float(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDeviceAngle)
            rt_plan_file['Beam'][beam_name]['dir_rot_colli'] = plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDeviceRotationDirection
            
            #rt_plan_file['Beam'][beam_name]['DOSE'] = float(plan.DoseReferenceSequence[0].DeliveryMaximumDose)
            #rt_plan_file['Beam'][beam_name]['UM'] = float(plan.FractionGroupSequence[0].ReferencedBeamSequence[3].BeamMeterset)
            #rt_plan_file['Beam'][beam_name]['Fraction'] = float(plan.FractionGroupSequence[0].NumberofFractionsPlanned)
            
            if rt_plan_file['Beam'][beam_name]['RadiationType'] == 'ELECTRON':
                rt_plan_file['Beam'][beam_name]['applicatorID'] = beam_.ApplicatorSequence[0].ApplicatorID
                rt_plan_file['Beam'][beam_name]['applicator_type'] = beam_.ApplicatorSequence[0].ApplicatorType
                rt_plan_file['Beam'][beam_name]['SSD'] = beam_.SourceAxisDistance
                rt_plan_file['Beam'][beam_name]['insert'] = beam_.BlockSequence[0].BlockName
            
            
            rt_plan_file['Beam'][beam_name]['ControlPointSequence']={}
            
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':             

                for control_point_sequence_ in beam_.ControlPointSequence:
                                 
                    control_point_index=control_point_sequence_.ControlPointIndex
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]={}
                    
                    if len(control_point_sequence_.BeamLimitingDevicePositionSequence)== 3:
                    
                        mlcx = [float(val) for val in control_point_sequence_.BeamLimitingDevicePositionSequence[2].LeafJawPositions]
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryRotationDirection'] = beam_.ControlPointSequence[0].GantryRotationDirection
                        xj = control_point_sequence_.BeamLimitingDevicePositionSequence[0].LeafJawPositions
                        yj= control_point_sequence_.BeamLimitingDevicePositionSequence[1].LeafJawPositions
                
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['x_jaw']=[float(val) for val in xj]
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['y_jaw']=[float(val) for val in yj]
               
                    else:
                        mlcx = [float(val) for val in control_point_sequence_.BeamLimitingDevicePositionSequence[0].LeafJawPositions]
                        xj = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].LeafJawPositions
                        yj= beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['x_jaw']=[float(val) for val in xj]
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['y_jaw']=[float(val) for val in yj]
                        
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['MLCx'] = [float(val) for val in mlcx]
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryAngle'] = round(float(control_point_sequence_.GantryAngle), 3)
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['CumulativeDoseCoef'] = round(float(control_point_sequence_.ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient), 4)
                    
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC':
                
                control_point_index=beam_.ControlPointSequence[0].ControlPointIndex
                rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]={}
                rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryAngle'] = round(float(beam_.ControlPointSequence[0].GantryAngle), 3)
                
                if len(beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence) > 2:
                
                    mlcx = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[2].LeafJawPositions
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['MLCx'] = [float(val) for val in mlcx]

#study=str(str(rt_plan_file['Name'])).split('^')[0]
study='test'
print("STUDY : " + study)
print("rt_plan_file dictionary  -->  Created")
print(rt_plan_file)

STUDY : test
rt_plan_file dictionary  -->  Created
{'Name': 'IM20190219', 'ID': 'IM20190219', 'Birth': '', 'Beam': {'Field 1': {'RadiationType': 'PHOTON', 'Energy': '6FFF', 'BeamType': 'DYNAMIC', 'x_jaw': [-27.5, 27.7], 'y_jaw': [-28.2, 29.3], 'iso': [0.2875261, 0.036824834, -0.0018387875], 'rot_colli': 0.0, 'dir_rot_colli': 'NONE', 'ControlPointSequence': {'0': {'GantryRotationDirection': 'CC', 'x_jaw': [-27.5, 27.7], 'y_jaw': [-28.2, 29.3], 'MLCx': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.4, -8.2, -2.0, -2.3, 4.3, 4.3, 4.3, -12.0, -12.0, -12.0, 0.4, -12.0, -12.0, -12.0, -12.0, -12.0, -3.1, 5.5, 4.3, -2.0, -12.0, -12.5, -12.0, -7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.4, 4.8, 10.8, 11.3, 10.8, 11.3, 10.8, 11.3, 11.3, 10.8, 12.3, 12.0, 12.0, 12.0, 10.8, 11.3, 10.8, 11.3, 11.3, 11.3, 11.3, 

## Write RT plan Summary ".txt"

In [54]:
rt_plan_sumup_name="/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/resume/rt_plan_"+ study +".txt"
rt_plan_sumup=open(rt_plan_sumup_name, 'w') 

rt_plan_sumup.write("*********************** RT PLAN SUMARY ******************************"+'\n') 
rt_plan_sumup.write('\n')
rt_plan_sumup.write('\n')

rt_plan_sumup.write("GENERAL INFORMATION :"  + '\n' + '\n')
rt_plan_sumup.write("     PATIENT Name : " + str(rt_plan_file['Name']) + '\n')                       #Write  patient name
rt_plan_sumup.write("     PATIENT ID : " + str(rt_plan_file['ID']) + '\n')                    #write patient ID
rt_plan_sumup.write("     PATIENT Birth Date : " + str(rt_plan_file['Birth']) + '\n' + '\n' + '\n')  

rt_plan_sumup.write("TECHNICAL INFORMATION :"  + '\n' + '\n')
rt_plan_sumup.write("     Number of Treatment Beams : " + str(len(rt_plan_file['Beam'])) + '\n' + '\n')

beamsname=rt_plan_file['Beam'].keys()
for beams in beamsname:
    rt_plan_sumup.write("     Beams Name : " + beams + '\n')
    rt_plan_sumup.write("          Beams Type : " + rt_plan_file['Beam'][beams]['BeamType'] + '\n')
    rt_plan_sumup.write("          Radiation Type : " + rt_plan_file['Beam'][beams]['RadiationType'] + '\n')
    rt_plan_sumup.write("          Nominal Energy : " + str(rt_plan_file['Beam'][beams]['Energy']) + '\n')
    rt_plan_sumup.write("          X jaws opening : " + str(rt_plan_file['Beam'][beams]['x_jaw']) + '\n')
    rt_plan_sumup.write("          Y jaws opening : " + str(rt_plan_file['Beam'][beams]['y_jaw']) + '\n')
    rt_plan_sumup.write("          Isocenter position : " + str(rt_plan_file['Beam'][beams]['iso']) + '\n')
    #rt_plan_sumup.write("          Dose prescription : " + str(rt_plan_file['Beam'][beams]['DOSE']) + '\n')
    #rt_plan_sumup.write("          UM (per fraction) : " + str(rt_plan_file['Beam'][beams]['UM']) + '\n')
    #rt_plan_sumup.write("          Fraction planned : " + str(rt_plan_file['Beam'][beams]['Fraction']) + '\n')

    
    
    if rt_plan_file['Beam'][beams]['RadiationType'] =='ELECTRON':
        rt_plan_sumup.write("          Applicator ID : " + rt_plan_file['Beam'][beam_name]['applicatorID'] + '\n')
        rt_plan_sumup.write("          Applicator type : " + rt_plan_file['Beam'][beam_name]['applicator_type'] + '\n')
        rt_plan_sumup.write("          INSERT : " + rt_plan_file['Beam'][beam_name]['insert'] + '\n')
        rt_plan_sumup.write("          DSP : " + str(rt_plan_file['Beam'][beam_name]['SSD']) + '\n')
            
    if rt_plan_file['Beam'][beams]['BeamType'] =='STATIC':
        
        rt_plan_sumup.write("          Gantry Angle : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][0]['GantryAngle']) + '\n')
        
    if rt_plan_file['Beam'][beams]['BeamType'] =='DYNAMIC':
        
        rt_plan_sumup.write("          Control Point index Number : " + str(len(rt_plan_file['Beam'][beams]['ControlPointSequence'])) + '\n')
        rt_plan_sumup.write("          GantryAngleRotation : " + rt_plan_file['Beam'][beams]['ControlPointSequence'][0]['GantryRotationDirection'] + '\n')
    
    
       # rt_plan_sumup.write('\n')

rt_plan_sumup.write(  '\n'  + '\n' + '\n' + '\n')
rt_plan_sumup.write("TECHNICAL INFORMATION DETAILED :"  + '\n' + '\n')


if rt_plan_file['Beam'][beams]['BeamType'] =='DYNAMIC' or len(beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence) > 2 :
    for beams in beamsname:
        rt_plan_sumup.write(" ===== Beams Name : " + beams + ' ======\n' + '\n')

        n_cpi=rt_plan_file['Beam'][beams]['ControlPointSequence'].keys()
        for cpi in n_cpi:
            rt_plan_sumup.write("   *** Control Point Index : " + str(cpi) + ' \n')
            rt_plan_sumup.write("      -> X Jaws : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['x_jaw']) + ' \n')
            rt_plan_sumup.write("      -> Y Jaws : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['y_jaw']) + ' \n')
            rt_plan_sumup.write("      -> MLCx : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['MLCx']) + ' \n')
            rt_plan_sumup.write("      -> Gantry Angle : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['GantryAngle']) + ' \n')
            if 'CumulativeDoseCoef' in rt_plan_file['Beam'][beams]['ControlPointSequence'][0]:
                rt_plan_sumup.write("      -> Cumul Dose Coeff : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['CumulativeDoseCoef']) + ' \n' + ' \n')
        rt_plan_sumup.write('\n')
    
rt_plan_sumup.write('\n')

rt_plan_sumup.close()
print("rt_plan_"+ study +'.txt' '  -->'  '  Created')
#print(rt_plan_file)

rt_plan_lung.txt  -->  Created


# Create list for dose rate

In [21]:
for beams_ in rt_plan_file['Beam'].keys():
    tot=0.0
    dose_rate_list=np.zeros(len(rt_plan_file['Beam'][beams_]['ControlPointSequence']))
    for i in range(len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])):
        if i !=0:
            n=(rt_plan_file['Beam'][beams_]['ControlPointSequence'][i]['CumulativeDoseCoef']-rt_plan_file['Beam'][beams_]['ControlPointSequence'][i-1]['CumulativeDoseCoef'])# IF YOU WANT IN UM *rt_plan_file['Beam'][beam_name]['UM']
            #print(n)
            dose_rate_list[i-1]=n
            tot=tot+n
    print('tot:', tot)
    dose_rate_list=dose_rate_list*len(dose_rate_list)
    print(dose_rate_list)
    #np.savetxt('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/dose_rate/dose_rate_'+study+'.dat' ,dose_rate_list)
    #np.savetxt('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/dose_rate/dose_rate_interplay_1D_FF_PTV10.dat' ,dose_rate_list, , fmt='%10.5f')
    np.savetxt('/media/sf_Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/interplay_1D_FF_PTV25/dose_rate/dose_rate_interplay_1D_FF_PTV25.dat' ,dose_rate_list, fmt='%10.5f')

tot: 1.0
[0.4806 0.4005 0.4806 0.4005 0.4806 0.4005 0.4806 0.801  0.8811 0.8811
 0.8811 0.8811 0.801  0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811
 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.9612 0.8811
 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811
 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811
 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.8811 0.9612 0.8811 0.8811
 0.9612 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811
 0.8811 0.9612 0.8811 0.8811 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811
 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811 0.9612
 0.8811 0.9612 0.8811 0.8811 0.8811 0.9612 0.8811 0.8811 0.9612 0.8811
 0.8811 0.9612 0.8811 0.8811 0.8811 0.9612 0.8811 0.9612 0.8811 0.9612
 0.8811 0.8811 0.9612 0.8811 0.801  0.801  0.8811 0.801  0.8811 0.801
 0.8811 0.801  0.8811 0.801  0.8811 0.801  0.801  0.8811 0.801  0.8811
 0.801  0.8811 0.801  0.8811 0.801  0.8811 0.801  0.801  0.8811 0.801

In [170]:
for beams_ in rt_plan_file['Beam'].keys():
    gantry_angle_list=np.zeros(len(rt_plan_file['Beam'][beams_]['ControlPointSequence']))
    for i in range(len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])):
        angle=rt_plan_file['Beam'][beams_]['ControlPointSequence'][i]['GantryAngle']
        gantry_angle_list[i]=angle
    print(gantry_angle_list)
    #np.savetxt('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/gantry_angle/gantry_angle_'+beams_+'_'+study+'.dat' ,gantry_angle_list, fmt='%10.5f')
    np.savetxt('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/gantry_angle/gantry_angle_'+study+'.dat' ,gantry_angle_list, fmt='%10.5f')


[ 90.     89.196  87.589  85.982  84.375  82.768  81.161  79.554  77.946
  76.339  74.732  73.125  71.518  69.911  68.304  66.696  65.089  63.482
  61.875  60.268  58.661  57.054  55.446  53.839  52.232  50.625  49.018
  47.411  45.804  44.196  42.589  40.982  39.375  37.768  36.161  34.554
  32.946  31.339  29.732  28.125  26.518  24.911  23.304  21.696  20.089
  18.482  16.875  15.268  13.661  12.054  10.446   8.839   7.232   5.625
   4.018   2.411   0.804 359.196 357.589 355.982 354.375 352.768 351.161
 349.554 347.946 346.339 344.732 343.125 341.518 339.911 338.304 336.696
 335.089 333.482 331.875 330.268 328.661 327.054 325.446 323.839 322.232
 320.625 319.018 317.411 315.804 314.196 312.589 310.982 309.375 307.768
 306.161 304.554 302.946 301.339 299.732 298.125 296.518 294.911 293.304
 291.696 290.089 288.482 286.875 285.268 283.661 282.054 280.446 278.839
 277.232 275.625 274.018 272.411 270.804 270.   ]


## Create mlc file for GATE

In [14]:
mlc_file_name='interplay.mlc'
for beams_ in rt_plan_file['Beam'].keys():
    #print beams_
    if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
        i=1
        for beams_ in rt_plan_file['Beam'].keys():
            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
                if rt_plan_file['Beam'][beams_]['BeamType'] == 'DYNAMIC':
                    #mlc_file_name='dynamicMLC_'+ beams_ + '_' + study + '.mlc'
                    mlc_file = open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/mlc/' + mlc_file_name, 'w')

                if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                    mlc_file_name='staticMLC_' + beams_ + '_' + study + '.mlc'
                    mlc_file = open(mlc_file_name, 'w')

                mlc_file.write('File Rev = H ' + '\n')
                mlc_file.write('Treatment = ' + rt_plan_file['Beam'][beams_]['BeamType'] + '\n')
                mlc_file.write('Last Name = ' + str(rt_plan_file['Name']) + '\n')
                mlc_file.write('First Name = ' + '\n')
                mlc_file.write('Patient ID = ' + str(rt_plan_file['ID']) + '\n')
                mlc_file.write('Number of Fields = ' + str(len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])) + '\n')
                mlc_file.write('Model = Varian HD120' + '\n')
                mlc_file.write('Tolerance = 0.00' + '\n')
                mlc_file.write('\n')

                j=0
                for cpi in rt_plan_file['Beam'][beams_]['ControlPointSequence']:


                    mlc_file.write('Field = ' + 'Beam ' + str(i)+'.'+str(j) + '\n')
                    if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                        mlc_file.write('Index = ' + str(rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['GantryAngle']) + '\n')
                    if rt_plan_file['Beam'][beams_]['BeamType'] == 'DYNAMIC':
                        mlc_file.write('Index = ' + str(rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['GantryAngle']) + '\n')            

                    mlc_file.write('Carriage Group = 1 ' + '\n')
                    mlc_file.write('Operator = ' + '\n')
                    mlc_file.write('Collimator = 0.00' + '\n')  
                    
                    listA=[]
                    listB=[]
                    
                    ii=0
                    for leafs_ in rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['MLCx']:

                        if ii < 60:
                            posa=round(((leafs_/10.0)*-1), 4)
                            mlc_file.write('Leaf ' + str(ii + 1) + 'A = ' + str(posa) + '\n')
                            #listB.append(leafs_)
                            #print('Leaf ' + str(ii + 1) + 'A = ' + str((leafs_/10.0)*-1) + '\n')
                        else:
                            #listA.append(leafs_)
                            posb=round((leafs_/10.0), 4)
                            mlc_file.write('Leaf ' + str(ii - 59) + 'B = ' + str(posb) + '\n')
                            #print('Leaf ' + str(ii - 59) + 'B = ' + str(leafs_/10.0) + '\n')                  

                        ii=ii+1                

                    mlc_file.write('Note = 0' + '\n')
                    mlc_file.write('Shape = 0' + '\n')
                    mlc_file.write('Magnification  = 1.00' + '\n' + '\n')            
                    j=j+1
            i=i+1
        print(mlc_file_name + ' --> created')    
        mlc_file.close()
    else:
        print('No mlc in this beam plan')
print(mlc_file_name)

interplay.mlc --> created
interplay.mlc


In [174]:
for beams_ in rt_plan_file['Beam'].keys():
    if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
        i=1
        for beams_ in rt_plan_file['Beam'].keys():
            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
                j=0
                for cpi in rt_plan_file['Beam'][beams_]['ControlPointSequence']:
                    
                                    
                    #mlc_file_name= str(j)+'-MLC_'+ beams_ + '_' + study + '.mlc'
                    #for interplay
                    mlc_file_name= str(j)+'-MLC_'+study+'.mlc'
                    #mlc_file = open('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/mlc/' +mlc_file_name, 'w')
                    mlc_file = open('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/interplay_1D_FF_PTV10/mlc2/' +mlc_file_name, 'w')

                    mlc_file.write('File Rev = H ' + '\n')
                    mlc_file.write('Treatment = ' + rt_plan_file['Beam'][beams_]['BeamType'] + '\n')
                    mlc_file.write('Last Name = ' + str(rt_plan_file['Name']) + '\n')
                    mlc_file.write('First Name = ' + '\n')
                    mlc_file.write('Patient ID = ' + str(rt_plan_file['ID']) + '\n')
                    mlc_file.write('Number of Fields = ' + str(len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])) + '\n')
                    mlc_file.write('Model = Varian HD120' + '\n')
                    mlc_file.write('Tolerance = 0.00' + '\n')
                    mlc_file.write('\n')
                    mlc_file.write('Field = ' + 'Beam ' + str(i)+'.'+str(j) + '\n')
                    mlc_file.write('Index = ' + str(rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['GantryAngle']) + '\n')            
                    mlc_file.write('Carriage Group = 1 ' + '\n')
                    mlc_file.write('Operator = ' + '\n')
                    mlc_file.write('Collimator = 0.00' + '\n')  
                    
                    ii=0
                    for leafs_ in rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['MLCx']:

                        if ii < 60:
                            mlc_file.write('Leaf ' + str(ii + 1) + 'A = ' + str((leafs_/10.0)*-1) + '\n')
                            #print('Leaf ' + str(ii + 1) + 'A = ' + str((leafs_/10.0)*-1) + '\n')
                        else:
                            mlc_file.write('Leaf ' + str(ii - 59) + 'B = ' + str(leafs_/10.0) + '\n')
                            #print('Leaf ' + str(ii - 59) + 'B = ' + str(leafs_/10.0) + '\n')                  

                        ii=ii+1                

                    mlc_file.write('Note = 0' + '\n')
                    mlc_file.write('Shape = 0' + '\n')
                    mlc_file.write('Magnification  = 1.00' + '\n' + '\n')
                    mlc_file.close()
                    print(mlc_file_name + ' --> created')
                    j=j+1            
            i=i+1
        
    else:
        print('No mlc in this beam plan')

0-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
1-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
2-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
3-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
4-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
5-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
6-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
7-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
8-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
9-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
10-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
11-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
12-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
13-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
14-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
15-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
16-MLC_interplay_1D_FF_PTV10_NoResampled.mlc --> created
17-MLC_interplay_1D_FF_PTV10_NoResampled.

## Create Clinic_Config.tb rt plan based
#### one file per beam

In [29]:
for beams_ in rt_plan_file['Beam'].keys():
    j=0
    for cpi in rt_plan_file['Beam'][beams_]['ControlPointSequence']:

        with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/clinic_config_temp.tb', 'r') as file:  #laod Config.tb   # Write JAW position in Config.tb file # At the moment only lfor the first Beam
            lines = file.readlines()

            x1_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][0])
            x2_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][1])
            y1_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][0])
            y2_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][1])

            if x1_val < 0:
                x1_val=int(x1_val*-1)
            if x2_val < 0:
                x2_val=int(x2_val*-1)
            if y1_val < 0:
                y1_val=int(y1_val*-1)
            if y2_val < 0:
                y2_val=int(y2_val*-1)

            lines[lines.index("=X1=\n")+1] = str(x1_val) +'\n'
            lines[lines.index("=X2=\n")+1] = str(x2_val) +'\n'
            lines[lines.index("=Y1=\n")+1] = str(y1_val) +'\n'
            lines[lines.index("=Y2=\n")+1] = str(y2_val) +'\n'
            
            lines[lines.index("=DURATION=\n")+1] = 'XXX'+'\n'
            lines[lines.index("=N_STEPS=\n")+1] = 'XXX'+'\n'

            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:   
                    lines[lines.index("=MLC=\n")+1] = '1' +'\n'
            else:
                lines[lines.index("=MLC=\n")+1] = '0' +'\n'

            lines[lines.index("=GANTRY_STOP=\n")+1] = 'X' +'\n'
            lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][j]['GantryAngle'])) +'\n'

            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
                lines[lines.index("=MLC_FILE=\n")+1]='MLC_'+ beams_ + '_' + study + '_'+ str(j) + '.mlc'+'\n'
            else:
                lines[lines.index("=MLC_FILE=\n")+1]='XXX' +'\n'

            lines[lines.index("=PHANTOM=\n")+1]=  '0' +'\n'

            lines[lines.index("=ISO=\n")+1]=  str(rt_plan_file['Beam'][beams_]['iso']).replace('[', '').replace(']', '')

            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
                a='E'
                lines[lines.index("=ACTOR=\n")+1]='phSpAirActorElec.mac' +'\n'
                lines[lines.index("=ENERGY=\n")+1]= str(rt_plan_file['Beam'][beams_]['Energy']) + a +'\n'
                
            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'PHOTON':
                #a='X'
                lines[lines.index("=ACTOR=\n")+1]='phSpAirActor.mac' +'\n'
                lines[lines.index("=ENERGY=\n")+1]= str(rt_plan_file['Beam'][beams_]['Energy']) + '\n'
                
            lines[lines.index("=INPUT_TYPE=\n")+1]='N' +'\n'
            lines[lines.index("=INPUT=\n")+1]='A' +'\n'
            lines[lines.index("=PHSP_R_NAME=\n")+1]='XXX' +'\n'


            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
                lines[lines.index("=APPLICATOR=\n")+1] = rt_plan_file['Beam'][beams_]['applicatorID'] +'\n'
                if rt_plan_file['Beam'][beams_]['insert'] == 'PERSO' or rt_plan_file['Beam'][beams_]['insert'] == 'perso':
                    lines[lines.index("=INSERT=\n")+1] = study+'.stl'
                else:
                    lines[lines.index("=INSERT=\n")+1] = rt_plan_file['Beam'][beams_]['applicator_type'].replace('ELECTRON_', '') +'\n'
            else:
                lines[lines.index("=APPLICATOR=\n")+1]='XXX' +'\n'
                lines[lines.index("=INSERT=\n")+1]='XXX' +'\n'

            lines[lines.index("=JOB_NAME=\n")+1] = str(beams_ +'_' +study +'_'+str(j))+'\n'
        
        with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/'+study+'/phsp/clinic_config_phsp_' + beams_ +'_' +study +'_'+ str(j) + '.tb', 'w') as file:                                                                                                      #Write Config.tb with changes
            file.writelines(lines)
            print('clinic_config_phsp_'+ beams_ + '_' + study +'_'+ str(j) +'.tb' ' -->'  '  Created') 
            j=j+1


clinic_config_phsp_ARCSTE_IMART_0.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_1.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_2.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_3.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_4.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_5.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_6.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_7.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_8.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_9.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_10.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_11.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_12.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_13.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_14.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_15.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_16.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_17.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_18.tb -->  Created
clinic_config_phsp_ARCSTE_IMART_19.tb -->

In [31]:
for beams_ in rt_plan_file['Beam'].keys():
    j=0
    for cpi in rt_plan_file['Beam'][beams_]['ControlPointSequence']:

        with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/clinic_config_temp.tb', 'r') as file:  #laod Config.tb   # Write JAW position in Config.tb file # At the moment only lfor the first Beam
            lines = file.readlines()

            x1_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][0])
            x2_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][1])
            y1_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][0])
            y2_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][1])

            if x1_val < 0:
                x1_val=int(x1_val*-1)
            if x2_val < 0:
                x2_val=int(x2_val*-1)
            if y1_val < 0:
                y1_val=int(y1_val*-1)
            if y2_val < 0:
                y2_val=int(y2_val*-1)

            lines[lines.index("=X1=\n")+1] = str(x1_val) +'\n'
            lines[lines.index("=X2=\n")+1] = str(x2_val) +'\n'
            lines[lines.index("=Y1=\n")+1] = str(y1_val) +'\n'
            lines[lines.index("=Y2=\n")+1] = str(y2_val) +'\n'
            
            lines[lines.index("=DURATION=\n")+1] = 'XXX'+'\n'
            lines[lines.index("=N_STEPS=\n")+1] = 'XXX'+'\n'

            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:   
                    lines[lines.index("=MLC=\n")+1] = '1' +'\n'
            else:
                lines[lines.index("=MLC=\n")+1] = '0' +'\n'

            lines[lines.index("=GANTRY_STOP=\n")+1] = 'X' +'\n'
            lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][j]['GantryAngle'])) +'\n'

            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
                lines[lines.index("=MLC_FILE=\n")+1]='MLC_'+ beams_ + '_' + study + '_'+ str(j) + '.mlc'+'\n'
            else:
                lines[lines.index("=MLC_FILE=\n")+1]='XXX' +'\n'

            lines[lines.index("=PHANTOM=\n")+1]=  study + '.mhd' +'\n'

            lines[lines.index("=ISO=\n")+1]=  str(rt_plan_file['Beam'][beams_]['iso']).replace('[', '').replace(']', '')

            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
                a='E'                
            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'PHOTON':
                #a='X'
                lines[lines.index("=ENERGY=\n")+1]= str(rt_plan_file['Beam'][beams_]['Energy']) + '\n'
                
            lines[lines.index("=ACTOR=\n")+1]= study +'_actor.mac' +'\n'

            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':    
                lines[lines.index("=INPUT_TYPE=\n")+1]='E' +'\n'
            else:
                lines[lines.index("=INPUT_TYPE=\n")+1]='M' +'\n'
               
            lines[lines.index("=INPUT=\n")+1]='A' +'\n'
            
            lines[lines.index("=PHSP_R_NAME=\n")+1]= study + '_phsp' +'\n'


            if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
                lines[lines.index("=APPLICATOR=\n")+1] = rt_plan_file['Beam'][beams_]['applicatorID'] +'\n'
                if rt_plan_file['Beam'][beams_]['insert'] == 'PERSO' or rt_plan_file['Beam'][beams_]['insert'] == 'perso':
                    lines[lines.index("=INSERT=\n")+1] = study+'.stl'
                else:
                    lines[lines.index("=INSERT=\n")+1] = rt_plan_file['Beam'][beams_]['applicator_type'].replace('ELECTRON_', '') +'\n'
            else:
                lines[lines.index("=APPLICATOR=\n")+1]='XXX' +'\n'
                lines[lines.index("=INSERT=\n")+1]='XXX' +'\n'

            lines[lines.index("=JOB_NAME=\n")+1] = str(beams_ +'_' +study +'_'+str(j))+'\n'
        
        with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/'+study+'/dose/clinic_config_dose_' + beams_ +'_' +study +'_'+ str(j) + '.tb', 'w') as file:                                                                                                      #Write Config.tb with changes
            file.writelines(lines)
            print('clinic_config_dose_'+ beams_ + '_' + study +'_'+ str(j) +'.tb' ' -->'  '  Created') 
            j=j+1

clinic_config_dose_ARCSTE_IMART_0.tb -->  Created
clinic_config_dose_ARCSTE_IMART_1.tb -->  Created
clinic_config_dose_ARCSTE_IMART_2.tb -->  Created
clinic_config_dose_ARCSTE_IMART_3.tb -->  Created
clinic_config_dose_ARCSTE_IMART_4.tb -->  Created
clinic_config_dose_ARCSTE_IMART_5.tb -->  Created
clinic_config_dose_ARCSTE_IMART_6.tb -->  Created
clinic_config_dose_ARCSTE_IMART_7.tb -->  Created
clinic_config_dose_ARCSTE_IMART_8.tb -->  Created
clinic_config_dose_ARCSTE_IMART_9.tb -->  Created
clinic_config_dose_ARCSTE_IMART_10.tb -->  Created
clinic_config_dose_ARCSTE_IMART_11.tb -->  Created
clinic_config_dose_ARCSTE_IMART_12.tb -->  Created
clinic_config_dose_ARCSTE_IMART_13.tb -->  Created
clinic_config_dose_ARCSTE_IMART_14.tb -->  Created
clinic_config_dose_ARCSTE_IMART_15.tb -->  Created
clinic_config_dose_ARCSTE_IMART_16.tb -->  Created
clinic_config_dose_ARCSTE_IMART_17.tb -->  Created
clinic_config_dose_ARCSTE_IMART_18.tb -->  Created
clinic_config_dose_ARCSTE_IMART_19.tb -->

# WITH DYNAMIC

In [18]:
for beams_ in rt_plan_file['Beam'].keys():

    with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/clinic_config_temp.tb', 'r') as file:                                  #laod Config.tb   # Write JAW position in Config.tb file # At the moment only lfor the first Beam
        lines = file.readlines()
         
        x1_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][0])
        x2_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][1])
        y1_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][0])
        y2_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][1])
        
        if x1_val < 0:
            x1_val=int(x1_val*-1)
        if x2_val < 0:
            x2_val=int(x2_val*-1)
        if y1_val < 0:
            y1_val=int(y1_val*-1)
        if y2_val < 0:
            y2_val=int(y2_val*-1)
                
        lines[lines.index("=X1=\n")+1] = str(x1_val) +'\n'
        lines[lines.index("=X2=\n")+1] = str(x2_val) +'\n'
        lines[lines.index("=Y1=\n")+1] = str(y1_val) +'\n'
        lines[lines.index("=Y2=\n")+1] = str(y2_val) +'\n'
        
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':
             lines[lines.index("=DURATION=\n")+1] = str(len(rt_plan_file['Beam'][beam_name]['ControlPointSequence'].keys()))+'\n'
             lines[lines.index("=N_STEPS=\n")+1] = str(int(len(rt_plan_file['Beam'][beam_name]['ControlPointSequence'].keys()))+1) +'\n'

                
        if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:   
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':        
                lines[lines.index("=MLC=\n")+1] = '2' +'\n'
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC' :
                lines[lines.index("=MLC=\n")+1] = '1' +'\n'
        else:
            lines[lines.index("=MLC=\n")+1] = '0' +'\n'
            
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC' :
                lines[lines.index("=GANTRY_STOP=\n")+1] = 'X' +'\n'
                lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryAngle'])) +'\n'
        
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC' :
            lines[lines.index("=GANTRY_STOP=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])-1]['GantryAngle'])) +'\n'
            lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryAngle'])) +'\n'
        
            if rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryRotationDirection'] == 'CW':
                lines[lines.index("=ROTATION_DIRECTION=\n")+1] = '0' +'\n'
            else:
                lines[lines.index("=ROTATION_DIRECTION=\n")+1] = '1' +'\n'
        
        if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
            if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                lines[lines.index("=MLC_FILE=\n")+1]='staticMLC_' + beams_ + '_' + study + '.mlc' +'\n'
            else:    
                lines[lines.index("=MLC_FILE=\n")+1]='dynamicMLC_' + beams_ + '_' + study + '.mlc' +'\n' 
        else:
            lines[lines.index("=MLC_FILE=\n")+1]='XXX' +'\n'
        
        lines[lines.index("=PHANTOM=\n")+1]= study + '.mhd' +'\n'
        
        lines[lines.index("=ISO=\n")+1]=  str(rt_plan_file['Beam'][beams_]['iso']).replace('[', '').replace(']', '')
        
        lines[lines.index("=ACTOR=\n")+1]='XXX' +'\n'
        
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
            a='E'
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'PHOTON':
            a='X'
        
        lines[lines.index("=ENERGY=\n")+1]= str(rt_plan_file['Beam'][beams_]['Energy']) + a +'\n'
        
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
            lines[lines.index("=APPLICATOR=\n")+1] = rt_plan_file['Beam'][beams_]['applicatorID'] +'\n'
            if rt_plan_file['Beam'][beams_]['insert'] == 'PERSO' or rt_plan_file['Beam'][beams_]['insert'] == 'perso':
                lines[lines.index("=INSERT=\n")+1] = study+'.stl'
            else:
                lines[lines.index("=INSERT=\n")+1] = rt_plan_file['Beam'][beams_]['applicator_type'].replace('ELECTRON_', '') +'\n'
            
        #print(beams_)
        lines[lines.index("=JOB_NAME=\n")+1] = beams_
        
    with open('/home/gate/Documents/phd/data/JupyterNotebook/RTplanReader/config/clinic_Config_' + beams_ +'_' +study + '.tb', 'w') as file:                                                                                                      #Write Config.tb with changes
        file.writelines(lines)
    print('ClinicConfig_'+ beams_ + '_' + study +'.tb' ' -->'  '  Created') 

ClinicConfig_Field 1_test.tb -->  Created
